## 缺失值的估算
由于各种原因，许多现实世界的数据集包含缺失值，通常将其编码为空白，NaN或其他占位符。但是，此类数据集与scikit-learn估计器不兼容，后者假定数组中的所有值都是数字，并且都具有并具有含义。使用不完整数据集的基本策略是丢弃包含缺失值的整个行和/或列。但是，这是以丢失有价值的数据为代价的（即使数据不完整）。更好的策略是估算缺失值，即从数据的已知部分推断出缺失值。参见 [插补通用术语表和API元素条目](https://scikit-learn.org/stable/glossary.html#glossary)。
### 单变量特征插补
**SimpleImputer,数据插补转换器**
 - missing_values:缺少数据所使用的占位符,默认为np.nan
 - strategy:填充缺失值的策略,默认为"mean",还有"median","most_frequent","constant;,均值与中位数只能与数字数据一起处理,频繁和常量(constant)可用于字符串和数字"
 - fill_value:数字或字符,默认为None;当strategy="constant"时,用于填充的内容;
 - verbose:默认=0
 - copy:默认为True
 - add_indicator:默认为False

In [1]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [2]:
X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
print(imp_mean.transform(X))

[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]]


### 多元特征插补(测试阶段,不稳定)
一种更复杂的方法是使用IterativeImputer类，,该类将缺失值的每个特征建模为其他特征的函数，并将该估计值用于插补。它以循环方式进行：在每个步骤中，将一个功能列指定为输出，y而将其他功能列视为输入X。一个回归器是众所周知的。然后，使用回归器预测的缺失值。针对每个特征以迭代方式完成此操作，然后重复进行插补回合。返回最后一轮估算的结果。

In [3]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit([[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]])

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [5]:
X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first
print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


## 最近邻特征插补

使用k-最近邻方法中缺失值进行填充。每个样本的缺失值都是使用n_neighbors训练集中找到的最近邻的平均值估算的 。如果两个都不缺失的特征都接近，则两个样本接近。
**KNNImputer**
 - missing_values:缺少数据所使用的占位符,默认为np.nan
 - n_neighbors:用于插补的相邻样本数,默认为5
 - weights:权重,默认uniform;uniform,均匀的权重,每个领域中的点均被加权;distance-距离,权重点按其距离的倒数来表示.在这种情况下,查询点的近邻比远处的近邻具有更大的影响力。
 - metric:距离度量,默认nan_euclidean:欧几里德距离
 - copy:如果为True，将创建X的副本。如果为False，则插补将在任何可能的地方进行。
 - add_indicator:默认为False

In [6]:
import numpy as np
from sklearn.impute import KNNImputer
X = [[1, 2, np.nan], [3, 4, 3], [np.nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(X)

array([[1. , 2. , 4. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])

###  标记推定值
**MissingIndicator**
二进制数据的缺失值补充方法
**参数：**
 - missing_values：数字,字符串，默认值 np.nan ，或者None;缺失值的占位符。 所有出现的missing_values都将被估算。 对于具有缺失值的可空整数dtypes的PandasDataFrame，missing_values应设置为np.nan，因为pd.NA将转换为np.nan。
 - features:字符串，默认为None；“missing-only” (default),表示只替换缺失数据；”all“表示替换所有数据
 - sparse:bool或者"auto",默认为None;掩码格式是稀疏的还是密集的,如果为"auto"则与输入类型相同.如果为True,则为稀疏,False:Imputer mask则为numpy数组
 - error_on_new:boolean,默认为None;如果为True（默认值），则当变换中的要素缺失值不适合缺失时，变换将引发错误。仅在时适用features="missing-only"。

In [1]:
import numpy as np
from sklearn.impute import MissingIndicator
X1 = np.array([[np.nan, 1, 3],
               [4, 0, np.nan],
               [8, 1, 0]])
X2 = np.array([[5, 1, np.nan],
               [np.nan, 2, 3],
               [2, 4, 0]])
indicator = MissingIndicator()
indicator.fit(X1)

MissingIndicator()

In [2]:
X2_tr = indicator.transform(X2)
X2_tr

array([[False,  True],
       [ True, False],
       [False, False]])